# CODIGO SIMULACION DE LOGS Y GRAFICO DE DATOS

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

Generamos datos 

Generar 500 registros de logs simulados

In [5]:
np.random.seed(42)

Lista de eventos de seguridad

In [6]:
eventos = ['login_success', 'login_failure', 'file_access', 'admin_login', 
           'password_change', 'firewall_block', 'suspicious_activity', 
           'malware_detected', 'port_scan']